In [1]:
import pandas as pd  
import numpy as np

**Data has been partially cleaned in Excel:**
- Removed blank 'Code X' columns
- Combined testing date columns into one, as all testing occurs at the same time.
- Combined examiner columns into one, , as all testing occurs at the same time.
- Removed 3 rows with no scoring data (2 are likely duplicates anyway)

In [2]:
evals_raw= pd.DataFrame(pd.read_excel('../data/bdi3_all_evals_clean.xlsx'))
evals_raw.head()

,Child ID,Gender,Date of Birth,Location - Sub Level 1,Program Label,Date of Testing,Examiner,Adaptive Sum of Scaled Scores,Adaptive Developmental Quotient,Adaptive Percentile Rank,...,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts SS,Cognitive-Perception and Concepts PR,Cognitive-Perception and Concepts AE,Cognitive-Perception and Concepts RDI,Cognitive-Perception and Concepts CSS,Cognitive-Perception and Concepts CSS 90%,Cognitive-Perception and Concepts Z-Score,Cognitive-Perception and Concepts T-Score,Cognitive-Perception and Concepts NCE
0,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden,2.0,46.0,<0.1,...,4.0,1.0,<1,2,<1/90,352.0,323 - 381,– 3.00,20.0,<1
1,404042,M,2020-03-17,Upper Cumberland,BDI-3 Annual Evaluation,2023-03-09,Stacy Holden,2.0,46.0,<0.1,...,4.0,1.0,<1,2,<1/90,352.0,323 - 381,– 3.00,20.0,<1
2,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges,2.0,46.0,<0.1,...,8.0,1.0,<1,8,<1/90,395.0,376 - 415,– 3.00,20.0,<1
3,475636,F,2019-10-16,Northwest,BDI-3 Annual Evaluation,2023-07-19,Jennifer Hodges,2.0,46.0,<0.1,...,8.0,1.0,<1,8,<1/90,395.0,376 - 415,– 3.00,20.0,<1
4,476193,M,2019-11-08,Memphis Delta,BDI-3 Annual Evaluation,2023-02-27,Kayla Coley,2.0,46.0,<0.1,...,8.0,1.0,<1,8,<1/90,395.0,376 - 415,– 3.00,20.0,<1


In [3]:
evals_raw[evals_raw['Child ID'].str.startswith('SP', na=False)]
evals_raw['is_duplicate'] = evals_raw.duplicated(subset=['Child ID', 'Date of Testing'])
evals = evals_raw[evals_raw['is_duplicate']==False]
evals = evals.drop('is_duplicate', axis=1)
evals.info()
evals.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 19435 entries, 0 to 23943
Columns: 186 entries, Child ID to Cognitive-Perception and Concepts NCE
dtypes: datetime64[ns](2), float64(69), object(115)
memory usage: 27.7+ MB


,Date of Birth,Date of Testing,Adaptive Sum of Scaled Scores,Adaptive Developmental Quotient,Adaptive T-Score,Social-Emotional Sum of Scaled Scores,Social-Emotional Developmental Quotient,Social-Emotional T-Score,Communication Sum of Scaled Scores,Communication Developmental Quotient,...,Cognitive-Attention and Memory CSS,Cognitive-Attention and Memory T-Score,Cognitive-Reasoning / Academic Skills RS,Cognitive-Reasoning / Academic Skills SS,Cognitive-Reasoning / Academic Skills CSS,Cognitive-Reasoning / Academic Skills T-Score,Cognitive-Perception and Concepts RS,Cognitive-Perception and Concepts SS,Cognitive-Perception and Concepts CSS,Cognitive-Perception and Concepts T-Score
count,19435,19435,19432.000000,19432.000000,19432.000000,19421.000000,19421.000000,19421.000000,19411.000000,19411.000000,...,19423.000000,19423.000000,9672.000000,9672.000000,9672.000000,9672.000000,19423.000000,19423.000000,19423.000000,19423.000000
mean,2021-08-25 17:28:38.404939520,2023-06-28 20:48:37.046565376,11.410457,87.414265,41.610848,22.380928,94.460532,46.308120,13.196126,79.588378,...,461.906245,44.701230,8.944065,8.098945,483.236042,43.632134,10.341914,7.784843,424.226021,42.618597
min,2019-10-15 00:00:00,2022-12-08 00:00:00,1.000000,46.000000,14.000000,2.000000,46.000000,14.000000,2.000000,46.000000,...,366.000000,20.000000,0.000000,1.000000,442.000000,20.000000,0.000000,1.000000,283.000000,20.000000
25%,2020-12-11 00:00:00,2023-04-21 00:00:00,7.000000,76.000000,34.000000,17.000000,84.000000,39.000000,8.000000,64.000000,...,452.000000,40.000000,6.000000,6.000000,475.000000,37.000000,8.000000,6.000000,395.000000,37.000000
50%,2021-07-21 00:00:00,2023-07-03 00:00:00,11.000000,88.000000,42.000000,22.000000,96.000000,47.000000,13.000000,79.000000,...,472.000000,47.000000,9.000000,8.000000,485.000000,43.000000,10.000000,8.000000,426.000000,43.000000
75%,2022-04-12 00:00:00,2023-09-08 12:00:00,15.000000,100.000000,50.000000,28.000000,106.000000,54.000000,18.000000,94.000000,...,482.000000,50.000000,12.000000,11.000000,493.000000,53.000000,12.000000,10.000000,451.000000,50.000000
max,2023-10-30 00:00:00,2023-11-15 00:00:00,36.000000,148.000000,82.000000,53.000000,154.000000,86.000000,38.000000,154.000000,...,544.000000,80.000000,30.000000,19.000000,543.000000,80.000000,48.000000,19.000000,554.000000,80.000000
std,NaN,NaN,5.821855,17.439723,11.613559,7.519675,16.701944,11.136879,6.723755,20.171265,...,32.563976,9.054627,4.207137,3.204089,14.151492,10.658037,4.227105,3.262026,43.754711,10.883636


There are 71 of 186 columns formatted in a way that allows for calculations.<br>Not all columns of scoring data are cleanly formatted; need to convert form type 'object' to a number format.

In [4]:
evals.columns.to_list()

['Child ID',
 'Gender',
 'Date of Birth',
 'Location - Sub Level 1',
 'Program Label',
 'Date of Testing',
 'Examiner',
 'Adaptive Sum of Scaled Scores',
 'Adaptive Developmental Quotient',
 'Adaptive Percentile Rank',
 'Adaptive 95% Confidence Interval',
 'Adaptive RDI',
 'Adaptive Z-Score',
 'Adaptive T-Score',
 'Adaptive NCE',
 'Social-Emotional Sum of Scaled Scores',
 'Social-Emotional Developmental Quotient',
 'Social-Emotional Percentile Rank',
 'Social-Emotional 95% Confidence Interval',
 'Social-Emotional RDI',
 'Social-Emotional Z-Score',
 'Social-Emotional T-Score',
 'Social-Emotional NCE',
 'Communication Sum of Scaled Scores',
 'Communication Developmental Quotient',
 'Communication Percentile Rank',
 'Communication 95% Confidence Interval',
 'Communication RDI',
 'Communication Z-Score',
 'Communication T-Score',
 'Communication NCE',
 'Motor Sum of Scaled Scores',
 'Motor Developmental Quotient',
 'Motor Percentile Rank',
 'Motor 95% Confidence Interval',
 'Motor RDI',
 '

In [5]:
def clean(column):
    months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    clean_col = evals[column].str[0:-3]
    clean_col = [months[i] if i in months.keys() else i for i in clean_col] 
    clean_col = [0 if str(i)[0:1] == '<' else i for i in clean_col] 
    clean_col = [100 if str(i)[0:1] == '>' else i for i in clean_col] 
    clean_col = pd.to_numeric(clean_col)
    return clean_col

In [6]:
for column in evals.columns:
    if 'RDI' in column:
        evals[column] = clean(column)

In [7]:
for column in evals.columns:
    if 'Percentile Rank' in column:
        evals[column] = clean(column)

In [8]:
for column in evals.columns:
    if 'NCE' in column:
        evals[column] = clean(column)

In [9]:
for column in evals.columns:
    if ' PR' in column:
        evals[column] = clean(column)

Add column: 'Age at Testing'

In [10]:
evals['Age at Testing'] = ((evals['Date of Testing'] - evals['Date of Birth']).dt.days) / 365

In [11]:
evals.value_counts('Program Label')

Program Label
BDI-3 Eligibility Evaluation          12342
BDI-3 Annual Evaluation                6890
BDI-3 Milestone or Exit Evaluation      203
Name: count, dtype: int64

To make things easier to work with, break the spreadsheet up into smaller .csv files.
- One for just the domains in aggregate (without subdomains) 

In [13]:
domain_keys = [
    'Child ID',
    'Gender',
    'Date of Birth',
    'Location - Sub Level 1',
    'Program Label',
    'Date of Testing',
    'Age at Testing'
    'Examiner',
    'Adaptive Sum of Scaled Scores',
    'Adaptive Developmental Quotient',
    'Adaptive Percentile Rank',
    'Adaptive 95% Confidence Interval',
    'Adaptive RDI',
    'Adaptive Z-Score',
    'Adaptive T-Score',
    'Adaptive NCE',
    'Social-Emotional Sum of Scaled Scores',
    'Social-Emotional Developmental Quotient',
    'Social-Emotional Percentile Rank',
    'Social-Emotional 95% Confidence Interval',
    'Social-Emotional RDI',
    'Social-Emotional Z-Score',
    'Social-Emotional T-Score',
    'Social-Emotional NCE',
    'Communication Sum of Scaled Scores',
    'Communication Developmental Quotient',
    'Communication Percentile Rank',
    'Communication 95% Confidence Interval',
    'Communication RDI',
    'Communication Z-Score',
    'Communication T-Score',
    'Communication NCE',
    'Motor Sum of Scaled Scores',
    'Motor Developmental Quotient',
    'Motor Percentile Rank',
    'Motor 95% Confidence Interval',
    'Motor RDI',
    'Motor Z-Score',
    'Motor T-Score',
    'Motor NCE',
    'Cognitive Sum of Scaled Scores',
    'Cognitive Developmental Quotient',
    'Cognitive Percentile Rank',
    'Cognitive 95% Confidence Interval',
    'Cognitive RDI',
    'Cognitive Z-Score',
    'Cognitive T-Score',
    'Cognitive NCE',
    'BDI-3 Total Sum of Scaled Scores',
    'BDI-3 Total Developmental Quotient',
    'BDI-3 Total Percentile Rank',
    'BDI-3 Total 90% Confidence Interval',
    'BDI-3 Total 95% Confidence Interval',
    'BDI-3 Total Z-Score',
    'BDI-3 Total T-Score',
    'BDI-3 Total NCE']
domain_level_scores = evals.loc[:, evals.columns.isin(domain_keys)]
domain_level_scores
domain_level_scores.to_csv('../data/domain_level_scores.csv')